# ⚡ 00.04: 고급 RAG 맛보기 (5분)

## 💡 기본 RAG를 더 똑똑하게 만드는 방법들!

In [ ]:
# 필수 라이브러리 설치
!pip install -q langchain-community faiss-cpu sentence-transformers rank-bm25

In [ ]:
# 1. 하이브리드 검색 맛보기 (Vector + BM25)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from rank_bm25 import BM25Okapi

documents = [
    "파이썬은 1991년 귀도가 만든 프로그래밍 언어입니다.",
    "머신러닝은 데이터로 패턴을 학습하는 기술입니다.",
    "RAG는 검색과 생성을 합친 AI입니다."
]

# BM25 검색 (키워드 기반)
tokenized_docs = [doc.split() for doc in documents]
bm25 = BM25Okapi(tokenized_docs)
query = "RAG 검색"
bm25_scores = bm25.get_scores(query.split())
print(f"🔤 BM25 검색 (키워드): {documents[bm25_scores.argmax()]}")

In [ ]:
# 2. 메타데이터 필터링 맛보기
docs_with_metadata = [
    Document(page_content="파이썬 프로그래밍 언어", metadata={"category": "언어", "year": 1991}),
    Document(page_content="머신러닝 기술", metadata={"category": "AI", "year": 2000}),
    Document(page_content="RAG 시스템", metadata={"category": "AI", "year": 2020})
]

# AI 카테고리만 필터링
ai_docs = [doc for doc in docs_with_metadata if doc.metadata["category"] == "AI"]
print(f"🏷️ AI 카테고리 문서 {len(ai_docs)}개 찾음:")
for doc in ai_docs:
    print(f"  - {doc.page_content} ({doc.metadata['year']}년)")

In [ ]:
# 3. 쿼리 확장 맛보기
def expand_query(query):
    """간단한 쿼리 확장"""
    synonyms = {
        "RAG": ["검색 증강 생성", "Retrieval Augmented Generation"],
        "AI": ["인공지능", "머신러닝", "딥러닝"],
        "파이썬": ["Python", "프로그래밍"]
    }
    expanded = [query]
    for word, syns in synonyms.items():
        if word in query:
            expanded.extend(syns)
    return expanded

original = "RAG 설명"
expanded = expand_query(original)
print(f"🔍 쿼리 확장:")
print(f"  원본: {original}")
print(f"  확장: {expanded}")

In [ ]:
# 4. 리랭킹 맛보기
import numpy as np

def simple_rerank(docs, query, metadata_boost=0.2):
    """간단한 리랭킹 (메타데이터 가중치 적용)"""
    scores = []
    for doc in docs:
        # 기본 점수 (랜덤하게 시뮬레이션)
        base_score = np.random.random()
        
        # 메타데이터 보너스
        bonus = 0
        if hasattr(doc, 'metadata'):
            if doc.metadata.get('category') == 'AI':
                bonus += metadata_boost
        
        scores.append(base_score + bonus)
    
    # 점수 순으로 정렬
    ranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
    return [doc for doc, score in ranked]

reranked = simple_rerank(docs_with_metadata, "AI 관련 질문")
print(f"🏆 리랭킹 결과:")
for i, doc in enumerate(reranked, 1):
    print(f"  {i}위: {doc.page_content} ({doc.metadata['category']})")

In [ ]:
# 5. 고급 RAG 기능들 정리
print("🎉 고급 RAG 맛보기 완료!")
print("✅ 하이브리드 검색: Vector + BM25 조합")
print("✅ 메타데이터 필터링: 카테고리별 필터")
print("✅ 쿼리 확장: 동의어로 검색 범위 확대")
print("✅ 리랭킹: 메타데이터로 우선순위 조정")
print("💡 다음: 00.05에서 RAG 성능 평가 맛보기!")